In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import unicodedata
from geopy.distance import geodesic, great_circle

In [2]:
def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)

In [3]:
flights = pd.read_csv('flightdiary20240507.csv', sep=',')
df = flights

In [4]:
df

,Date,Flight number,From,To,Dep time,Arr time,Duration,Airline,Aircraft,Registration,Seat number,Seat type,Flight class,Flight reason,Note,Dep_id,Arr_id,Airline_id,Aircraft_id
0,2022-06-28,FR1604,Kerkira / Ioannis Kapodistrias (CFU/LGKR),Marseille / Marseille Provence (MRS/LFML),09:40:00,10:55:00,02:15:00,Ryanair (FR/RYR),Boeing 737-800 (B738),EI-EKO,4A,1,0,0,NaN,531,1928,668,231
1,2022-06-25,FR7444,Marseille / Marseille Provence (MRS/LFML),Kerkira / Ioannis Kapodistrias (CFU/LGKR),10:25:00,13:30:00,02:05:00,Malta Air (AL/MAY),Boeing 737-800 (B738),9H-QDI,4A,1,0,0,NaN,1928,531,2723,231
2,2022-05-17,U23062,Nice / Nice Cote D'Azur (NCE/LFMN),Paris / Charles de Gaulle (CDG/LFPG),06:15:00,07:55:00,01:40:00,easyJet Europe (EC/EJU),Airbus A320-200 (A320),OE-ICP,8E,2,0,0,NaN,2027,509,2665,2059
3,2022-03-18,W68299,Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),Nice / Nice Cote D'Azur (NCE/LFMN),16:30:00,17:50:00,01:20:00,Wizz Air (W6/WZZ),Airbus A321neo (A21N),NaN,NaN,0,0,0,NaN,872,2027,859,2128
4,2022-03-18,EK97,Dubai / Dubai International (DXB/OMDB),Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),09:10:00,13:25:00,06:15:00,Emirates (EK/UAE),Boeing 777-300ER (B77W),A6-EGS,NaN,0,0,0,NaN,768,872,807,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,2025-01-16,AZ64,Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),Madrid / Barajas (MAD/LEMD),21:25:00,23:59:00,02:34:00,ITA Airways (AZ/ITY),Airbus A320neo (A20N),NaN,NaN,0,0,0,NaN,872,1739,2967,2127
230,2024-11-11,GA409,Denpasar Bali / Ngurah Rai (DPS/WADD),Jakarta / Soekarno Hatta (CGK/WIII),13:45:00,14:45:00,02:00:00,Garuda Indonesia (GA/GIA),Airbus A330-200 (A332),NaN,NaN,0,0,0,NaN,3582,534,327,30
231,2024-06-16,MF8086,Dalian / Zhoushuizi (DLC/ZYTL),Nanjing / Nanjing (NKG/ZSNJ),09:05:00,10:50:00,01:45:00,Xiamen Airlines (MF/CXA),Boeing 737-800 (B738),NaN,NaN,0,0,0,STOPOVER,720,2066,215,231
232,2024-06-16,MF8086,Nanjing / Nanjing (NKG/ZSNJ),Fuzhou / Fuzhou (FOC/ZSFZ),12:05:00,13:40:00,01:35:00,Xiamen Airlines (MF/CXA),Boeing 737-800 (B738),NaN,NaN,0,0,0,STOPOVER,2066,909,215,231


In [5]:
keys_x = df.keys()
keys_x

Index(['Date', 'Flight number', 'From', 'To', 'Dep time', 'Arr time',
       'Duration', 'Airline', 'Aircraft', 'Registration', 'Seat number',
       'Seat type', 'Flight class', 'Flight reason', 'Note', 'Dep_id',
       'Arr_id', 'Airline_id', 'Aircraft_id'],
      dtype='object')

In [6]:
def get_code(string, start, length):
  match = re.search(r'\((.*?)\)', string)

  if match:
      content_inside_brackets = match.group(1)
      first_length_chars = content_inside_brackets[start:start+length]

  return first_length_chars

In [7]:
def get_airline_code(airline):
  if get_code(airline, 0, 2)[0] == "/": return get_code(airline, 1, 3)
  return get_code(airline, 0, 2)

In [8]:
def get_airport_code(airport):
  return get_code(airport, 0, 3)

In [9]:
# df_result = pd.DataFrame(columns=keys_x)
# df_result

In [10]:
# find out the flights that didn't operate by its iata code airline and its number
# not include in the final output file
count = 0
table = []
for i in range(df.shape[0]):
  if (not (df.iloc[i][keys_x[1]][:2] in [])) and (df.iloc[i][keys_x[1]][:2] != get_airline_code(df.iloc[i][keys_x[7]])):
    count += 1
    double = [df.iloc[i][keys_x[1]][:2], df.iloc[i][keys_x[7]]]
    triple = [df.iloc[i][keys_x[0]], df.iloc[i][keys_x[1]][:2], df.iloc[i][keys_x[7]]]
    if not double in table:
      table.append(double)
print(count)
for i in range(len(table)):
  print(table[i])

66
['FR', 'Malta Air (AL/MAY)']
['U2', 'easyJet Europe (EC/EJU)']
['FR', 'Lauda Air (NG/LDA)']
['U2', 'Easyjet Switzerland (DS/EZS)']
['FR', 'Buzz (RR/RYS)']
['UX', 'CanaryFly (PM/CNF)']
['CA', 'Dalian Airlines (/CCD)']
['AF', 'KLM Royal Dutch Airlines (KL/KLM)']
['KL', 'China Southern Airlines (CZ/CSN)']
['U2', 'SmartLynx Airlines (6Y/ART)']
['IB', 'Air Nostrum (YW/ANE)']
['BT', 'Avion Express Malta (/MLH)']
['KL', 'KLM Cityhopper (WA/KLC)']
['CZ', 'Chongqing Airlines (OQ/CQN)']
['AF', 'HOP! (A5/HOP)']


In [11]:
# compute iata code for dep/arr airport for each flight
df_from = df['From']
df_to = df['To']
from_to_pair = []
for i in range(df.shape[0]):
  pair = [get_airport_code(df_from[i]), get_airport_code(df_to[i])]
  from_to_pair.append(pair)
assert(len(from_to_pair) == df.shape[0])
print(from_to_pair)

[['CFU', 'MRS'], ['MRS', 'CFU'], ['NCE', 'CDG'], ['FCO', 'NCE'], ['DXB', 'FCO'], ['BKK', 'DXB'], ['HKG', 'BKK'], ['HEL', 'HKG'], ['NCE', 'HEL'], ['VIE', 'MRS'], ['NCE', 'VIE'], ['FCO', 'NCE'], ['CTA', 'FCO'], ['FCO', 'CTA'], ['NCE', 'FCO'], ['TFS', 'NCE'], ['BSL', 'TFS'], ['NCE', 'BSL'], ['HBE', 'MXP'], ['CAI', 'ASW'], ['MXP', 'HBE'], ['BDS', 'MXP'], ['FCO', 'BDS'], ['TFS', 'FCO'], ['PMI', 'TFN'], ['MXP', 'PMI'], ['LCA', 'MXP'], ['AMM', 'PFO'], ['WMI', 'AMM'], ['KBP', 'WMI'], ['WRO', 'LWO'], ['OHD', 'VIE'], ['CIA', 'TIA'], ['CTA', 'FCO'], ['NCE', 'CTA'], ['CDG', 'NCE'], ['NCE', 'ORY'], ['CDG', 'NCE'], ['GVA', 'NCE'], ['LIS', 'ZRH'], ['NTE', 'LIS'], ['NCE', 'ORY'], ['VIE', 'NCE'], ['CPH', 'MUC'], ['NCE', 'ORY'], ['TFS', 'MXP'], ['LPA', 'TFN'], ['MXP', 'LPA'], ['WMI', 'BGY'], ['MXP', 'KRK'], ['LIS', 'MXP'], ['MXP', 'LIS'], ['VIE', 'NCE'], ['CRL', 'BUD'], ['NCE', 'BRU'], ['FCO', 'NCE'], ['MLA', 'NAP'], ['ATH', 'CIA'], ['FCO', 'MLA'], ['JTR', 'ATH'], ['ATH', 'JTR'], ['BGY', 'ATH'], ['SOF',

In [12]:
# airline alliance
# 0: skyteam
skyteam = ["AR", "AM", "UX", "AF", "CI", "MU", "OK", "DL", "GA", "AZ", "KQ", "KL", "KE", "ME", "SV", "RO", "VN", "VS", "MF"]
# skyteam.append("SU")
skyteam.append("A5") # AF affiliate
skyteam.append("FM") # MU affiliate
skyteam.append("WA") # KL affiliate
# 1: staralliance
staralliance = ["A3", "AC", "CA", "AI", "NZ", "NH", "OZ", "OS", "AV", "SN", "CM", "OU", "MS", "ET", "BR", "LO", "LH", "SK", "ZH", "SQ", "SA", "LX", "TP", "TG", "TK", "UA"]
staralliance.append("CCD") # CA affiliate
# 2: oneworld
oneworld = ["AS", "AA", "BA", "CX", "AY", "IB", "JL", "MH", "QF", "QR", "AT", "RJ", "UL"]
oneworld.append("FY") # MH affiliate
oneworld.append("YW") # IB affiliate

In [13]:
alliance_ref_dict = {}
df_airline = df['Airline']
airline_uniq = np.unique(df_airline)
for i in range(len(airline_uniq)):
  iata_code = get_airline_code(airline_uniq[i])
  found = False
  if (not found) and (iata_code in skyteam):
    alliance_ref_dict.update({iata_code: 1})
    found = True
  if (not found) and (iata_code in staralliance):
    alliance_ref_dict.update({iata_code: 2})
    found = True
  if (not found) and (iata_code in oneworld):
    alliance_ref_dict.update({iata_code: 3})
    found = True
  if (not found): alliance_ref_dict.update({iata_code: 0})
print(alliance_ref_dict)

{'SU': 0, 'AK': 0, 'KC': 0, 'CA': 2, 'AF': 1, 'YW': 3, 'MLH': 0, 'PG': 0, 'BA': 3, 'RR': 0, 'PM': 0, 'CX': 3, 'MU': 1, 'CZ': 0, 'OQ': 0, 'CCD': 2, 'U2': 0, 'DS': 0, 'MS': 2, 'EK': 0, 'EY': 0, 'AY': 3, 'FY': 3, 'GA': 1, 'A5': 1, 'AZ': 1, 'JL': 3, 'WA': 1, 'KL': 1, 'KQ': 1, 'NG': 0, 'JT': 0, 'Z9': 0, 'MH': 3, 'AL': 0, 'DD': 0, 'DY': 0, 'D8': 0, 'DV': 0, 'QW': 0, 'FR': 0, 'SV': 1, 'TR': 0, 'FM': 1, 'GQ': 0, '6Y': 0, 'TP': 2, 'FD': 0, 'VZ': 0, 'GS': 0, 'HV': 0, 'TO': 0, 'TK': 2, 'VN': 1, 'V7': 0, 'VY': 0, 'W6': 0, '5W': 0, 'W4': 0, 'MF': 1, 'ZG': 0, 'EC': 0}


In [14]:
df_alliance_index = []
for airline in df_airline:
  if alliance_ref_dict.get(get_airline_code(airline)) != None: df_alliance_index.append(alliance_ref_dict.get(get_airline_code(airline)))
  else: df_alliance_index.append(alliance_ref_dict.get(0))
print(df_alliance_index)

[0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 3, 3, 1, 0, 3, 3, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 1, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 3, 0, 1, 1, 1, 1, 0, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
iata_icao = pd.read_csv('iata-icao.csv', sep=',')
df_iata = iata_icao

In [16]:
df_iata

,country_code,region_name,iata,icao,airport,latitude,longitude
0,AE,Abu Zaby,AAN,OMAL,Al Ain International Airport,24.2617,55.6092
1,AE,Abu Zaby,AUH,OMAA,Abu Dhabi International Airport,24.4330,54.6511
2,AE,Abu Zaby,AYM,NaN,Yas Island Seaplane Base,24.4670,54.6103
3,AE,Abu Zaby,AZI,OMAD,Al Bateen Executive Airport,24.4283,54.4581
4,AE,Abu Zaby,DHF,OMAM,Al Dhafra Air Base,24.2482,54.5477
...,...,...,...,...,...,...,...
8932,ZW,Masvingo,MVZ,FVMV,Masvingo Airport,-20.0553,30.8591
8933,ZW,Matabeleland North,HWN,FVWN,Hwange National Park Airport,-18.6299,27.0210
8934,ZW,Matabeleland North,VFA,FVFA,Victoria Falls Airport,-18.0959,25.8390
8935,ZW,Matabeleland North,WKI,FVWT,Hwange Town Airport,-18.3630,26.5198


In [17]:
# addtional airports whose gps is missing
bgy = [45.673889, 9.704166]
bsl = [47.590000, 7.529167]
cia = [41.799361, 12.594936]
itm = [34.785528, 135.438222]
pkx = [39.509167, 116.410556]
sai = [13.369167, 104.223056]
sxf = [52.378333, 13.520556] # same with BER?
tfn = [28.482653, -16.341535]
tfs = [28.044475, -16.572488]

In [18]:
# match airports with its gps
airport_uniq = np.unique(np.reshape(from_to_pair, df.shape[0] * 2))
airport_coor = []
missing_count = 0
print("!" + "-" * 32 + "!")
for airport in airport_uniq:
  coordinates = df_iata[df_iata['iata'] == airport]
  if coordinates.size != 0:
    airport_coor_pair = [airport, [coordinates.iloc[0][-2], coordinates.iloc[0][-1]]]
    airport_coor.append(airport_coor_pair)
  else:
    print("!", missing_count, airport, "coordinates data missing !")
    # airport_coor.append([airport])
    if (airport == 'BGY'):
      airport_coor.append([airport, bgy])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'BSL'):
      airport_coor.append([airport, bsl])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'CIA'):
      airport_coor.append([airport, cia])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'ITM'):
      airport_coor.append([airport, itm])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'PKX'):
      airport_coor.append([airport, pkx])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'SAI'):
      airport_coor.append([airport, sai])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'SXF'):
      airport_coor.append([airport, sxf])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'TFN'):
      airport_coor.append([airport, tfn])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    elif (airport == 'TFS'):
      airport_coor.append([airport, tfs])
      print("!", missing_count, airport, "added " + "-" * 19 + "!")
    missing_count += 1
print("!" + "-" * 32 + "!")
assert(len(airport_coor) == len(airport_uniq))
# airport_coor

!--------------------------------!
! 0 BGY coordinates data missing !
! 0 BGY added -------------------!
! 1 BSL coordinates data missing !
! 1 BSL added -------------------!
! 2 CIA coordinates data missing !
! 2 CIA added -------------------!
! 3 ITM coordinates data missing !
! 3 ITM added -------------------!
! 4 PKX coordinates data missing !
! 4 PKX added -------------------!
! 5 SAI coordinates data missing !
! 5 SAI added -------------------!
! 6 SXF coordinates data missing !
! 6 SXF added -------------------!
! 7 TFN coordinates data missing !
! 7 TFN added -------------------!
! 8 TFS coordinates data missing !
! 8 TFS added -------------------!
!--------------------------------!


C:\Users\admin\AppData\Local\Temp\ipykernel_20056\2255837039.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  airport_coor_pair = [airport, [coordinates.iloc[0][-2], coordinates.iloc[0][-1]]]


In [19]:
# make sure each airport has its gps
line_counter = 0
miss_counter = 0
for i in airport_coor:
  if len(i) == 1:
    miss_counter += 1
    print(line_counter, i)
  line_counter += 1
if (miss_counter == 0): print("Airport coordinates completed!")

Airport coordinates completed!


In [20]:
# geodesic(bkk, dxb).km
# great_circle(bkk, dxb).km

In [21]:
# compute triplet [[from, [from_lat, from_lon]], [to, [to_lat, to_lon], dist]
from_to_pair_coor = []
for pair in from_to_pair:
  pair_coor = []
  for airport in pair:
    triple = airport_coor[np.where(airport_uniq == airport)[0][0]]
    if len(triple) == 2: pair_coor.append(triple)
    else: print(airport, "coordinates missing")
  pair_coor.append(round(geodesic(pair_coor[0][1], pair_coor[1][1]).km)) # calculate dist with geodesic
  from_to_pair_coor.append(pair_coor)
assert(len(from_to_pair_coor) == df.shape[0])
print(df.shape[0], "flight routes completed!")

234 flight routes completed!


In [22]:
from_to_pair_coor

[[['CFU', [39.6019, 19.9117]], ['MRS', [43.4393, 5.22142]], 1296],
 [['MRS', [43.4393, 5.22142]], ['CFU', [39.6019, 19.9117]], 1296],
 [['NCE', [43.6584, 7.21587]], ['CDG', [49.0097, 2.54778]], 695],
 [['FCO', [41.8003, 12.2389]], ['NCE', [43.6584, 7.21587]], 460],
 [['DXB', [25.2528, 55.3644]], ['FCO', [41.8003, 12.2389]], 4349],
 [['BKK', [13.6811, 100.747]], ['DXB', [25.2528, 55.3644]], 4909],
 [['HKG', [22.3089, 113.915]], ['BKK', [13.6811, 100.747]], 1689],
 [['HEL', [60.3172, 24.9633]], ['HKG', [22.3089, 113.915]], 7821],
 [['NCE', [43.6584, 7.21587]], ['HEL', [60.3172, 24.9633]], 2203],
 [['VIE', [48.1103, 16.5697]], ['MRS', [43.4393, 5.22142]], 1022],
 [['NCE', [43.6584, 7.21587]], ['VIE', [48.1103, 16.5697]], 878],
 [['FCO', [41.8003, 12.2389]], ['NCE', [43.6584, 7.21587]], 460],
 [['CTA', [37.4668, 15.0664]], ['FCO', [41.8003, 12.2389]], 539],
 [['FCO', [41.8003, 12.2389]], ['CTA', [37.4668, 15.0664]], 539],
 [['NCE', [43.6584, 7.21587]], ['FCO', [41.8003, 12.2389]], 460],
 [

In [23]:
# prepare for output
df_enrich = df
df_enrich.insert(3, "Iata_from", [pair[0][0] for pair in from_to_pair_coor], True)
df_enrich.insert(4, "From_lat", [pair[0][1][0] for pair in from_to_pair_coor], True)
df_enrich.insert(5, "From_lon", [pair[0][1][1] for pair in from_to_pair_coor], True)
df_enrich.insert(7, "Iata_to", [pair[1][0] for pair in from_to_pair_coor], True)
df_enrich.insert(8, "To_lat", [pair[1][1][0] for pair in from_to_pair_coor], True)
df_enrich.insert(9, "To_lon", [pair[1][1][1] for pair in from_to_pair_coor], True)
df_enrich.insert(10, "Dist", [pair[2] for pair in from_to_pair_coor], True)
df_enrich.insert(15, "Alliance", df_alliance_index, True)
df_enrich

,Date,Flight number,From,Iata_from,From_lat,From_lon,To,Iata_to,To_lat,To_lon,...,Registration,Seat number,Seat type,Flight class,Flight reason,Note,Dep_id,Arr_id,Airline_id,Aircraft_id
0,2022-06-28,FR1604,Kerkira / Ioannis Kapodistrias (CFU/LGKR),CFU,39.60190,19.91170,Marseille / Marseille Provence (MRS/LFML),MRS,43.43930,5.22142,...,EI-EKO,4A,1,0,0,NaN,531,1928,668,231
1,2022-06-25,FR7444,Marseille / Marseille Provence (MRS/LFML),MRS,43.43930,5.22142,Kerkira / Ioannis Kapodistrias (CFU/LGKR),CFU,39.60190,19.91170,...,9H-QDI,4A,1,0,0,NaN,1928,531,2723,231
2,2022-05-17,U23062,Nice / Nice Cote D'Azur (NCE/LFMN),NCE,43.65840,7.21587,Paris / Charles de Gaulle (CDG/LFPG),CDG,49.00970,2.54778,...,OE-ICP,8E,2,0,0,NaN,2027,509,2665,2059
3,2022-03-18,W68299,Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),FCO,41.80030,12.23890,Nice / Nice Cote D'Azur (NCE/LFMN),NCE,43.65840,7.21587,...,NaN,NaN,0,0,0,NaN,872,2027,859,2128
4,2022-03-18,EK97,Dubai / Dubai International (DXB/OMDB),DXB,25.25280,55.36440,Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),FCO,41.80030,12.23890,...,A6-EGS,NaN,0,0,0,NaN,768,872,807,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,2025-01-16,AZ64,Rome / Leonardo da Vinci-Fiumicino (FCO/LIRF),FCO,41.80030,12.23890,Madrid / Barajas (MAD/LEMD),MAD,40.47190,-3.56264,...,NaN,NaN,0,0,0,NaN,872,1739,2967,2127
230,2024-11-11,GA409,Denpasar Bali / Ngurah Rai (DPS/WADD),DPS,-8.74817,115.16700,Jakarta / Soekarno Hatta (CGK/WIII),CGK,-6.12556,106.65600,...,NaN,NaN,0,0,0,NaN,3582,534,327,30
231,2024-06-16,MF8086,Dalian / Zhoushuizi (DLC/ZYTL),DLC,38.96570,121.53900,Nanjing / Nanjing (NKG/ZSNJ),NKG,31.74200,118.86200,...,NaN,NaN,0,0,0,STOPOVER,720,2066,215,231
232,2024-06-16,MF8086,Nanjing / Nanjing (NKG/ZSNJ),NKG,31.74200,118.86200,Fuzhou / Fuzhou (FOC/ZSFZ),FOC,25.93510,119.66300,...,NaN,NaN,0,0,0,STOPOVER,2066,909,215,231


In [24]:
pd.DataFrame.to_csv(df_enrich, '../js/data/flightdiary.csv', sep=',')

## City

In [25]:
visited_cities_csv = pd.read_csv('TravelCities_Y.csv', sep=';')
visited_cities = visited_cities_csv
visited_cities

,city,iso3,FREQ,1st arrival,1st arrival transport,2nd arrival,2nd arrival transport,3rd arrival,3rd arrival transport,4th arrival,4th arrival transport
0,Dalian,CHN,1,1997-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Shenyang,CHN,1,2000-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yantai (Penglai),CHN,1,2009-07-01,S,NaN,NaN,NaN,NaN,NaN,NaN
3,Tai'an,CHN,1,2009-07-01,B,NaN,NaN,NaN,NaN,NaN,NaN
4,Qingdao,CHN,1,2009-07-01,B,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
150,Amsterdam,NLD,2,2018-01-26,A,2024-01-28,F,NaN,NaN,NaN,NaN
151,Victoria (Mahe),SYC,1,2024-01-29,F,NaN,NaN,NaN,NaN,NaN,NaN
152,The Hague (Den Haag),NLD,1,2020-02-24,T,NaN,NaN,NaN,NaN,NaN,NaN
153,Genoa,ITA,1,2022-04-15,T,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
visited_iso3_uniq = np.unique(list(visited_cities.loc[:, "iso3"]))
visited_iso3_uniq

array(['ALB', 'ARE', 'ARM', 'AUT', 'AZE', 'BEL', 'BGR', 'BIH', 'CHE',
       'CHN', 'CYP', 'CZE', 'DEU', 'DNK', 'EGY', 'ESP', 'EST', 'FIN',
       'FRA', 'GBR', 'GEO', 'GRC', 'HKG', 'HRV', 'HUN', 'IDN', 'IRL',
       'IRN', 'ITA', 'JOR', 'JPN', 'KAZ', 'KHM', 'KOR', 'MAC', 'MAR',
       'MCO', 'MKD', 'MLT', 'MYS', 'NLD', 'OMN', 'POL', 'PRT', 'ROU',
       'RUS', 'SGP', 'SMR', 'SRB', 'SVK', 'SVN', 'SWE', 'SYC', 'THA',
       'TUR', 'UKR', 'UZB', 'VAT'], dtype='<U3')

In [27]:
keys_visited_cities = visited_cities.keys()
keys_visited_cities

Index(['city', 'iso3', 'FREQ', '1st arrival', '1st arrival transport',
       '2nd arrival', '2nd arrival transport', '3rd arrival',
       '3rd arrival transport', '4th arrival', '4th arrival transport'],
      dtype='object')

In [28]:
world_cities_csv = pd.read_csv('worldcities.csv', sep=',')
# cities = world_cities_csv[world_cities_csv["capital"].notnull()]
cities = world_cities_csv[world_cities_csv["iso3"].isin(visited_iso3_uniq)] # filter to have less data to accelerate the match

In [29]:
cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
6,Shanghai,Shanghai,31.2286,121.4747,China,CN,CHN,Shanghai,admin,24073000.0,1156073548
8,Seoul,Seoul,37.5600,126.9900,"Korea, South",KR,KOR,Seoul,primary,23016000.0,1410836482
...,...,...,...,...,...,...,...,...,...,...,...
47851,Īlām,Ilam,33.6374,46.4227,Iran,IR,IRN,Īlām,admin,NaN,1364655025
47860,Pailin,Pailin,12.8489,102.6093,Cambodia,KH,KHM,Pailin,admin,NaN,1116050327
47861,Ta Khmau,Ta Khmau,11.4833,104.9500,Cambodia,KH,KHM,Kandal,admin,NaN,1116571704
47865,Muan,Muan,34.9897,126.4714,"Korea, South",KR,KOR,Jeonnam,admin,NaN,1410001061


In [30]:
cities_simplify = cities.loc[:, ["city_ascii", "iso3", "lat", "lng"]]
cities_simplify = cities_simplify.rename(columns={"city_ascii": "city"})
cities_simplify.reset_index(drop=True, inplace=True)
cities_simplify

,city,iso3,lat,lng
0,Tokyo,JPN,35.6897,139.6922
1,Jakarta,IDN,-6.1750,106.8275
2,Guangzhou,CHN,23.1300,113.2600
3,Shanghai,CHN,31.2286,121.4747
4,Seoul,KOR,37.5600,126.9900
...,...,...,...,...
18499,Ilam,IRN,33.6374,46.4227
18500,Pailin,KHM,12.8489,102.6093
18501,Ta Khmau,KHM,11.4833,104.9500
18502,Muan,KOR,34.9897,126.4714


In [31]:
# additional city with its gps
extra_city = [["Villfranche-sur-Mer", [43.705, 7.3125]],
              ["Eze", [43.72861, 7.36167]],
              ["Merzouga", [31.09917, 4.01167]],
              ["Schwangau", [47.58306, 10.73306]],
              ["Santorini", [36.38806, 25.45972]],
              ["Wadi Musa", [30.32, 35.4783]],
              ["Abu Simbel", [22.33694, 31.62556]]]

In [32]:
# long runtime here
city_gps = []
for i in range(len(visited_cities)):
  city = visited_cities.loc[i, [keys_visited_cities[0], keys_visited_cities[1]]]
  find = False
  for j in range(len(cities_simplify)):
    city_simp = cities_simplify.loc[j, ["city", "iso3"]]
    city_half = city.iloc[0]
    if (city.iloc[0].find("(") != -1): city_half = city.iloc[0][:city.iloc[0].find("(") - 1]
    if (city_simp.iloc[0] == strip_accents(city_half)) and (city_simp.iloc[1] == city.iloc[1]):
      find = True
      city_gps.append([cities_simplify.loc[j, "lat"], cities_simplify.loc[j, "lng"]])
      break
  if (not find):
    print("!", i, city.iloc[0], city.iloc[1], "GPS data missing!")
    for pair in extra_city:
      if (strip_accents(city_half) == pair[0]):
        city_gps.append([pair[1][0], pair[1][1]])
        print("!", i, city.iloc[0], city.iloc[1], "Added ----------!")

! 12 Villfranche-sur-Mer FRA GPS data missing!
! 12 Villfranche-sur-Mer FRA Added ----------!
! 13 Èze FRA GPS data missing!
! 13 Èze FRA Added ----------!
! 47 Merzouga (Sahara) MAR GPS data missing!
! 47 Merzouga (Sahara) MAR Added ----------!
! 48 Schwangau DEU GPS data missing!
! 48 Schwangau DEU Added ----------!
! 91 Santorini (Fira) GRC GPS data missing!
! 91 Santorini (Fira) GRC Added ----------!
! 103 Wadi Musa JOR GPS data missing!
! 103 Wadi Musa JOR Added ----------!
! 107 Abu Simbel EGY GPS data missing!
! 107 Abu Simbel EGY Added ----------!


In [33]:
# prepare for output
visited_cities.insert(2, "lat", [pair[0] for pair in city_gps], True)
visited_cities.insert(3, "lon", [pair[1] for pair in city_gps], True)
visited_cities

,city,iso3,lat,lon,FREQ,1st arrival,1st arrival transport,2nd arrival,2nd arrival transport,3rd arrival,3rd arrival transport,4th arrival,4th arrival transport
0,Dalian,CHN,38.9000,121.6000,1,1997-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Shenyang,CHN,41.8025,123.4281,1,2000-03-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yantai (Penglai),CHN,37.4646,121.4478,1,2009-07-01,S,NaN,NaN,NaN,NaN,NaN,NaN
3,Tai'an,CHN,36.2020,117.0870,1,2009-07-01,B,NaN,NaN,NaN,NaN,NaN,NaN
4,Qingdao,CHN,36.0669,120.3827,1,2009-07-01,B,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Amsterdam,NLD,52.3728,4.8936,2,2018-01-26,A,2024-01-28,F,NaN,NaN,NaN,NaN
151,Victoria (Mahe),SYC,-4.6231,55.4525,1,2024-01-29,F,NaN,NaN,NaN,NaN,NaN,NaN
152,The Hague (Den Haag),NLD,52.0800,4.3100,1,2020-02-24,T,NaN,NaN,NaN,NaN,NaN,NaN
153,Genoa,ITA,44.4111,8.9328,1,2022-04-15,T,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pd.DataFrame.to_csv(visited_cities, '../js/data/travelcities.csv', sep=',')